In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 3615

# 1. Path

In [ ]:
paths = {
    "유형": ["results/type/2022-12-22_205334/checkpoint-1856","kykim/electra-kor-base"],
    "극성": ["results/polarity/2022-12-22_223149/checkpoint-696","kykim/electra-kor-base"],
    "시제": ["results/tense/2022-12-22_205421/checkpoint-464","kykim/electra-kor-base"],
    "확실성": ["results/certainty/2022-12-22_223359/checkpoint-464","kykim/electra-kor-base"],
}

# 2. Data

In [ ]:
!cp ../Dacon_Ryan/dataset/test.csv dataset/result.csv

In [ ]:
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.notebook import tqdm
from datasets import load_dataset

tds = load_dataset(
    "csv",
    data_files={"predict": "dataset/result.csv"}
)

In [ ]:
def inference(model, tokenizer, category):
    pipe = pipeline("text-classification", model=model, tokenizer = tokenizer, device=0)
    result = []
    for out in tqdm(pipe(KeyDataset(tds["predict"], "문장"), batch_size=64)):
        result.append(out["label"])
    return result

In [ ]:
for category, (pretrained_model_name_or_path, plm) in paths.items():
    tokenizer = AutoTokenizer.from_pretrained(plm)
    model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path)
    result = inference(model, tokenizer, category)
    tds["predict"] = tds["predict"].add_column(name=category, column=result)

In [ ]:
label = ["-".join(x) for x in zip(tds["predict"]["유형"], tds["predict"]["극성"], tds["predict"]["시제"], tds["predict"]["확실성"])]
tds["predict"] = tds["predict"].add_column(name="label", column=label)
# tds["predict"] = tds["predict"].map(lambda: "-".join(batch["유형"], batch["극성"], batch["시제"], batch["확신성"], remove_columns=list(paths.keys())))

In [ ]:
tds["predict"][1]

In [ ]:
tds['predict'][1]

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict({'ID': tds['predict']['ID'], 'label' : tds['predict']['label']})
df.head()

In [ ]:
df

In [ ]:
df.to_csv("submission_ryan.csv", encoding="utf-8-sig", index=False)